In [ ]:
import random as r
import numpy as np
import keras.layers as kl
import keras.models as km

In [ ]:
def next_frame(last_step, last_frame, column):
    lower = max(0, last_step - 1)
    upper = min(last_frame.shape[0] - 1, last_step + 1)

    step = r.randint(lower, upper)

    frame = last_frame.copy()
    frame[step, column] = 1
    
    return frame, step

In [ ]:
def build_frames(size):
    frames = list()

    frame = np.zeros((size, size))
    step = r.randint(0, size - 1)

    right = 1 if r.random() < 0.5 else 0
    col = 0 if right else size - 1

    frame[step, col] = 1
    frames.append(frame)

    for i in range(1, size):
        col = i if right else size-1-i
        frame, step = next_frame(step, frame, col)
        frames.append(frame)
        
    return frames, right

In [ ]:
def generate_examples(size, n_patterns):
    X, y = list(), list()
    for _ in range(n_patterns):
        frames, right = build_frames(size)
        X.append(frames)
        y.append(right)
    
    X = np.array(X).reshape(n_patterns, size, size, size, 1)
    y = np.array(y).reshape(n_patterns, 1)

    return X, y

In [ ]:
size = 50

model = km.Sequential()
model.add(kl.TimeDistributed(kl.Conv2D(2, (2, 2), activation='relu'), input_shape=(None, size, size, 1)))
model.add(kl.TimeDistributed(kl.Flatten()))
model.add(kl.LSTM(50))
model.add(kl.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
X, y = generate_examples(size, 5000)
model.fit(X, y, epochs=1, batch_size=32)

In [ ]:
X, y = generate_examples(size, 100)
loss, acc = model.evaluate(X, y, verbose=0)
print('loss: %f, acc: %f' % (loss, acc * 100))

In [ ]:
X, y = generate_examples(size, 1)
yhat = model.predict(X, verbose=0)
expected = "Right" if y[0] == 1 else "Left"
predicted = "Right" if yhat[0] == 1 else "Left"
print('Expected: %s, Predicted: %s' % (expected, predicted))